Option chains
=======

In [4]:
from ib_insync import *

from datetime import datetime, timezone, timedelta

import pandas as pd


In [5]:
underlying_symbol='CL'

#underlying filters
months_forward = 3
min_option_open_interest = 100000
min_iv_rank = .10 #10% = .10

#option strategy filters
pct_under_px_range = .20 # scan strikes under this % below cunderlying price 
num_month_expiries = 3 #expires forward
max_risk = 2000 # max loss
min_profit = 200 #max profit
ask_tws_load =True #ask wheter to load strategies

We want to find high ROC credit bull PUT verticals for future contracts, in 'underlying_symbol' underlyings that comply with :

* future contracts for months forward up to 'months_forward'
* Average option volume > 'min_option_volume'.
* Iv rank > min_iv_rank

After previous filter, scan option chains for remaining underlyings:

* Explore up to 'num_month_expiries' monthly option expiries forward from current date.
* Scan combinations of strike prices starting from smallest strike to  (ATM - 'pct_under_px_range'% ) range. Zero explores all available strike combinations.
* Limit risk to 'max_risk' USD (which influences allowable distance between strikes).

Notes:
1. If notional value of a naked put strategy is less than 'max_risk' then switch to naked put strategy (do not buy 2long leg).
2. Order resulting strategies descending on 'Return on Capital' (MaxProfit / Margin ).
3. If 'ask_tws_load==True', user will be prompted to load strategies on TWS.
4. Only OTM options will be scaned (strikes below current price)


In [6]:
util.startLoop()

ib = IB()
ib.connect('127.0.0.1', 7497, clientId=15, readonly = True)

<IB connected to 127.0.0.1:7497 clientId=15>

**Search future contracts on 'underlying_symbol' commodity with filters: 'months_forward', 'min_option_volume', 'min_iv_rank'**

In [7]:
#get 'months_forward' month expiration datetime
curdate = datetime.now()
three_months = timedelta( weeks=months_forward*4 )
expiration_limit = curdate + three_months

expiration_limit

datetime.datetime(2020, 8, 18, 13, 44, 10, 788492)

In [8]:
contracts = []

contract = Future(symbol=underlying_symbol, exchange='NYMEX', currency='USD')

contracts = ib.reqContractDetails(contract)

print('{} contracts found related to {}'.format( len(contracts), underlying_symbol ) )

128 contracts found related to CL


**Filter by months_forward**

In [9]:
filteredContracts = []

for c in contracts:
    contract_exp = pd.to_datetime( c.contract.lastTradeDateOrContractMonth,  format='%Y%m%d' )
    if contract_exp < expiration_limit:
        filteredContracts.append(c.contract)

print( '{} contracts after months_forward = {} filter'.format( len( filteredContracts ), months_forward ) )

filteredContracts

2 contracts after months_forward = 3 filter


[Contract(secType='FUT', conId=174230633, symbol='CL', lastTradeDateOrContractMonth='20200721', multiplier='1000', exchange='NYMEX', currency='USD', localSymbol='CLQ0', tradingClass='CL'),
 Contract(secType='FUT', conId=174230636, symbol='CL', lastTradeDateOrContractMonth='20200622', multiplier='1000', exchange='NYMEX', currency='USD', localSymbol='CLN0', tradingClass='CL')]

**Filter by minimum open interest**

This needs market data subscription.

In [10]:
tickers = {}

#generickTickList:
# 101: call option open interest, put option open interest
# 105: avOptionVolume (not used, not working)
# 106: impliedvolatility

#TODO: this might fail for larger contract lists 
for c in filteredContracts:
    tickers.update( { c.conId : ib.reqMktData( c, genericTickList='101,105,106' ) } )

#wait a couple of seconds for ticker info to be filled
ib.sleep( 2 )

True

Get sum call + put open interest and filter

In [11]:
for c in filteredContracts:
    ticker = tickers[ c.conId ]
    if (ticker.putOpenInterest + ticker.callOpenInterest) < min_option_open_interest:
        filteredContracts.remove( ticker.contract )
        ib.cancelMktData( ticker.contract )
        tickers.remove( c.conId )
        
print( '{} contracts after min_option_open_interest = {} '.format( len(filteredContracts), min_option_open_interest ) )

2 contracts after min_option_open_interest = 100000 


**Filter by IV rank**

In [12]:
#get IV historical bars

histBars = {}

for c in filteredContracts:
    bars = ib.reqHistoricalData(
            c,
            endDateTime='',
            durationStr='256 D',
            barSizeSetting='1 day',
            whatToShow='OPTION_IMPLIED_VOLATILITY',
            useRTH=True,
            formatDate=1)
    
    histBars.update( { c.conId : bars} )
    
ib.sleep(2)


True

In [13]:
#highest 53 W IV
ivHighs = {}
#current IV
ivCurrents = {}
#curent 52 W IV rank
ivRanks = {}

#get highest IV / IV rank
for c in filteredContracts:
    curHistBars = histBars[ c.conId ]
    sortedBars = sorted( curHistBars, key=lambda bar: bar.close, reverse=True )
    ivHighs.update( { c.conId : sortedBars[0].close } )
    ivCurrents.update( { c.conId : curHistBars[-1].close } )
    print( 'IV rank for {} contract: {}'.format( c.conId, curHistBars[-1].close / sortedBars[0].close ) )
    ivRanks.update( { c.conId : curHistBars[-1].close / sortedBars[0].close } )
    


IV rank for 174230633 contract: 0.3014510358321232
IV rank for 174230636 contract: 0.23303539570299792


In [14]:
for c in filteredContracts:
    if ivRanks[ c.conId ] < min_iv_rank:
        filteredContracts.remove(c)
        tickers.remove( c.conId )
        ib.cancelMktData( c )
        
        
print( '{} contracts surpased iv rank minumum if {}'.format(len(filteredContracts), min_iv_rank ) )
            

2 contracts surpased iv rank minumum if 0.1


**At this point screener has discriminated future contracts by:**

    * Expiration.
    * Open Interest (Put+Call Option contracts).
    * 52 week IV rank.

**Request option chains for remaining contracts**

In [15]:
chains = {}

for c in filteredContracts:
    chains.update( { c.conId : ib.reqSecDefOptParams(c.symbol, c.exchange, c.secType, c.conId ) } )

print('{} chains found'.format( len(chains) ) )

#util.df(chains)

2 chains found


From this point on we can build all PUT  option contracts that meet our conditions (Naked puts / Bull Put Spreads).

Detail of implemented algorithm:
  
1. Iterate over stock option 'chains', select first chain.
  1. filter the expirations list, limit to up to 3 month expirations.
  2. filter strike list, limit to strikes below 'pct_under_px_range'
  3. Explore remaining strike / expiriation combinations in filtered option chains:
     1. **Expirations loop**. Iterate over expirations, select first expiration (might be only one per chain):
        1.  **Leg 1 loop**. Iterate over strikes, select leg 1 strike.
            1. Combine selected strike with current expiration from outer loop.
            2. Build Option contract.
            3. Request market data (price) for 1st leg ontract.
            4. Create combo leg 1. Fix for going to inner loop.
            5. Cancel market data (if not we would reach ticker limit), enter inner loop.
                 1. **Leg 2 loop**. Iterate over strikes, select leg 2 strike
                     1. Is strike < combo leg 1 strike? No : Discard / iterate again, Yes: Select it.
                     2. Combine selected strike with current expiration from outer loop
                     3. Build Option contract.
                     4. Request market data (price) for 2nd leg contract.
                     5. Cancel market data.
                     6. Create 'COMBO' order,  request 'whatif' for price / margin information.
                     7. Calculate max profit, max loss, ROC from price / margin result.
                     8. Is maximum profit > 'min_profit' from screener parameters? No: Discard.
                     9. Is maximum loss > 'max_risk' from screener parameters? Yes: Discard.
                     10. Is resulting ROC better than previous iteration? No: Discard, Yes: Save as 'l3RocWinner'.
                     11. Continue loop to step  1 in **Leg 2 loop**
            6. Is 'l3RocWinner' ROC better than last 'l2RocWinner'?  Yes, Save as 'l2RocWinner'
            7. Save last 'l3RocWinner' in 'L3RocWinners'.
            8. Continue loop to step 1 in **Leg 1 loop**.
        2. Is 'l2RocWinner' ROC better than last 'l1RocWinner'? Yes, save as 'l1RocWinner'.
        3. Continue loop to step 1 in **Expirations loop**
  4. Results are stored in 'RocWinners' variables:
     * L3RocWinners: List of best Roc combos per short leg strike
     * L2RocWinners: List of best Roc combos per expiration
     * L1RocWinner: Best strategy overall          

In [16]:
# prepare 'num_month_expiries' datetime object
curdate = datetime.now()
delta_forward = timedelta( weeks = num_month_expiries*4 )
option_expiration_limit = curdate + delta_forward

option_expiration_limit

datetime.datetime(2020, 8, 18, 13, 44, 17, 664140)

**Filter option chains by strikes / epxirations**

In [17]:
# filter chains strikes / expirations:

adjustedContractChains = {}

for c in filteredContracts:
    adjustedChains = []
    curChains = chains[ c.conId  ]
    curPrice = tickers[ c.conId ].marketPrice() # might need to replace by 'saved' value
    print( 'current price for contract {} : {}'.format( c.conId, curPrice ) )
    
    for chain in curChains:
        
        expdate = pd.to_datetime( chain.expirations[ 0 ]  )
        
        # expiration within range? (suppose sorted already)
        if( expdate < option_expiration_limit ):
            #filter strikes ('pct_under_px_range')
            newstrikes = [ s for s in chain.strikes if s > ( curPrice * ( 1 - pct_under_px_range ) ) and s <= curPrice ]
                        
            #OptionChain is 'NamedTuple' (not mutable)
            adjustedChain = OptionChain( chain.exchange, 
                                        chain.underlyingConId,
                                        chain.tradingClass,
                                        chain.multiplier,
                                        newstrikes,
                                        chain.expirations )
            #adjustedChain.strikes = newstrikes
            adjustedChains.append( adjustedChain )
    
    adjustedContractChains.update( { c.conId : adjustedChains } )

current price for contract 174230633 : 34.78
current price for contract 174230636 : 34.34


**Create L1/L2 combo orders, calculate prices, filter by min_profit, max_loss**

In [66]:
# screened contracts will be placed here, one list per contract
# { conId : { combo : orderState } }
screenedCombos = {}

# Create / Filter comboss
for c in filteredContracts:

    for chain in adjustedContractChains[ c.conId ]:
        
            #combo orders for current contract
            comboOrders = []
            
            #suppose only 1 expiration per chain
            comboContracts = [Option(c.symbol, chain.expiration[0], strike, 'P', c.exchange, chain.expirations[0] )
                    for strike in chain.strikes]

            comboContracts = ib.qualifyContracts( *comboContracts )
            
            # should be less than 100 requests (request limit)
            [ comboTickers ] = ib.reqTickers( *comboContracts )

            #wait for tickers to be filled 
            ib.sleep(2)
            
            print( '{} tickers received out of {} for combo leg 1 contracts'.format( ) )
            
            # ---- leg1 loop ----
            for leg1Ticker in comboTickers:
                leg1Price = leg1Ticker.marketPrice()
                leg1Strike = leg1Ticker.contract.strike
                 # early filter leg1 by min_profit
                if( leg1Price < min_profit ):
                    ib.cancelMktData( ticker.contract )
                    comboContracts.remove(ticker.contract)
                else:
                    #fix leg 1
                    leg1 = ComboLeg(conId=leg1Ticker.contract.conId , 
                                    ratio= 1, 
                                    action='SELL', 
                                    exchange=leg1Ticker.contract.exchange )
                    # ---- leg 2 loop ----
                    for leg2Ticker in comboTickers:
                        leg2Strike = leg2Ticker.contract.strike
                        if( leg2Ticker.contract.strike < leg1Strike ):
                            leg2Price = leg2Ticker.marketPrice()
                            #leg2 hedge should cost less than leg1
                            comboPrice = leg1Price - leg2Price
                            if ( leg1Strike - leg2Strike ) * chain.multiplier < max_loss and
                               ( comboPrice  * chain.multiplier ) > min_profit:
                                    
                                    leg2 = ComboLeg( conId=leg2Ticker.contract.conId,  
                                                    ratio=1,
                                                    action='BUY',
                                                    exchange=leg2Ticker.contract.exchange )
                                    
                                    #create 'combo contract'
                                    combo = Contract( symbol=leg1Ticker.ticker.symbol,#check!! 
                                                     secType='BAG', 
                                                     currency='USD', 
                                                     exchange=leg1Ticker.contract.exchange, 
                                                     comboLegs=[leg1, leg2])
                                    
                                    order = LimitOrder( action="BUY", 
                                                       totalQuantity = 1, 
                                                       lmtPrice= comboPrice )
                                    
                                    
                                    orderState = ib.whatIfOrder( combo, order )
                                    
                                    #should be able to get leg details via ib.qualifyContracts
                                    comboOrders.update( { combo : orderState } )
                                    
            print( '{} combos found for contract id : {}'.format( len(comboOrders), c.conId ) )
            screenedCombos.update( { c.conId: comboOrders } )

24

The option greeks are available from the ``modelGreeks`` attribute, and if there is a bid, ask resp. last price available also from ``bidGreeks``, ``askGreeks`` and ``lastGreeks``. For streaming ticks the greek values will be kept up to date to the current market situation.

In [70]:
print(legOneTicker.hasBidAsk())

print(legOneTicker.lastGreeks)

True
OptionComputation(impliedVol=0.2996510031175316, delta=-0.2505361266671315, optPrice=1.4199999570846558, pvDividend=0.0, gamma=0.03425643599844212, vega=0.09614354874490338, theta=-0.3672188996639118, undPrice=296.030029296875)


build first combo leg

In [71]:
comboLegOne = ComboLeg(conId=legOne.conId, ratio= 1, action='SELL', exchange='SMART' )


Select second put contract as second leg

In [72]:
legTwo = putContracts[16]

[ legTwoTicker ] = ib.reqTickers( legTwo )

legTwo

Option(conId=415769250, symbol='SPY', lastTradeDateOrContractMonth='20200522', strike=280.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPY   200522P00280000', tradingClass='SPY')

In [84]:

print( 'leg two ask', legTwoTicker.ask )

print( 'leg two bid', legTwoTicker.bid )

print( 'leg one ask', legOneTicker.ask )

print( 'leg two ask', legOneTicker.bid )


leg two ask 0.36
leg two bid 0.35
leg one ask 1.43
leg two ask 1.42


The option greeks are available from the ``modelGreeks`` attribute, and if there is a bid, ask resp. last price available also from ``bidGreeks``, ``askGreeks`` and ``lastGreeks``. For streaming ticks the greek values will be kept up to date to the current market situation.

In [75]:
print( legTwoTicker.hasBidAsk() )

print(legTwoTicker.lastGreeks)

True
OptionComputation(impliedVol=0.3656978235015641, delta=-0.07013157712085653, optPrice=0.36000001430511475, pvDividend=0.0, gamma=0.011849231376148921, vega=0.03895589495174473, theta=-0.1896778776512994, undPrice=296.0400085449219)


Build second combo leg

In [76]:
comboLegTwo = ComboLeg(conId=legTwo.conId, ratio=1, action='BUY', exchange='SMART')

In [77]:
combo = Contract( symbol=spx.symbol, secType='BAG', currency='USD', exchange='SMART', comboLegs=[comboLegOne, comboLegTwo] )


Submit the order, do not transmit, get order price / margin information (use 'whatif' flag)... this might need market data

In [51]:
aproxPrice = legOneTicker.last - legTwoTicker.last

#get 'whatif'
order = LimitOrder( action="BUY", totalQuantity = 1, lmtPrice= aproxPrice )


#this will have the effect of adding the order to
# tws 'Activity' pane
comboStatus = ib.whatIfOrder( combo, order )


In [78]:
comboStatus

OrderState(status='PreSubmitted', initMarginBefore='3615.67', maintMarginBefore='3615.67', equityWithLoanBefore='1019599.0', initMarginChange='1000.0', maintMarginChange='1000.0', equityWithLoanChange='-119.18999999994412', initMarginAfter='4615.67', maintMarginAfter='4615.67', equityWithLoanAfter='1019479.81', commission=1.7976931348623157e+308, minCommission=1.293038, maxCommission=3.193038, commissionCurrency='USD', warningText='', completedTime='', completedStatus='')

In [80]:
#'verify price'
IbOrder = LimitOrder( action='BUY', totalQuantity=1, transmit=False, lmtPrice=aproxPrice)

trade = ib.placeOrder( combo, IbOrder )


In [11]:
ib.disconnect()